# Restaurant atmosphere/vibes analysis

I found some apps, that contain images of restaurants along with a vibe labelled.

`Tripadvisor` only has `business meetings`, `romantic`, and ideal for `groups vibes`. So it is not enough.

After searching a looot of apps, I found the `One Zone app`. Desgined specifically to chose a restaurant by the vibe they evoke. However, the dataset is really small, as they add restaurants manually. They don't use any kind of deep learning method. Maybe we can use it as a test dataset.

Therefore, what I am going to try to do, is to develop a NLP model to classify customer reviews by the restaurant vibe. Then I would label the restaurants images by the vibe obtained with the NLP model.

Yelp Dataset

So we'll be working with Yelp dataset. The Yelp Open Dataset is an extensive collection provided by Yelp for academic and research purposes. It offers a rich source of data from Yelp's vast repository of local business reviews and user interactions. It contains:

1. **Businesses:**
Basic information about local businesses, including their name, location (address, city, state, postal code), latitude and longitude, average rating, category (e.g., Restaurants, Shopping, Beauty & Spas), and the number of reviews they have received. This allows for analyses focused on geographical trends, category-based studies, and more.

2. **Reviews:**
Millions of text reviews written by Yelp users for various businesses. Each review includes the user's text review, the star rating they gave, and the date of the review. This data is invaluable for sentiment analysis, natural language processing tasks, and understanding consumer preferences.

3. **Users:**
Information about Yelp users who have written reviews, including their user ID, name, review count, yelping since (the date they joined Yelp), friends, and other social metrics. This can be used for social network analysis, studying user behavior, and personalization algorithms.

4. **Check-ins:**
Data about check-ins at businesses by Yelp users, which can help in analyzing foot traffic trends, popular times, and loyalty or frequency of visits.

5. **Tips:**
Short notes left by users about a business. Tips can contain advice, recommendations, or comments about what's good or what to avoid. This dataset is useful for extracting quick insights or highlights about businesses.

6. **Photos:**
Metadata about photos associated with businesses, including a photo ID, business ID, caption, and labels indicating the photo category (e.g., food, interior, exterior). This can support visual analysis of businesses or complement review-based insights as in our case.

## CSV File prepared

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# EDA

In [4]:
import pandas as pd

In [5]:
df = pd.read_csv("/content/drive/MyDrive/restaurant_reviews_with_photos.csv")

<ipython-input-5-9d5eea62fb33>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/drive/MyDrive/restaurant_reviews_with_photos.csv")


In [6]:
df.head()

,business_id,text,photo_id
0,XQfwVwDr-v0ZS3_CbbE5Xw,"If you decide to eat here, just be aware it is...",NaN
1,YjUWPpI6HXG530lwP-fb2A,Family diner. Had the buffet. Eclectic assortm...,NaN
2,kxX2SOes4o-D3ZQBkiMRfA,"Wow! Yummy, different, delicious. Our favo...",NaN
3,e4Vwtrqf-wpJfwesgvdgxQ,Cute interior and owner (?) gave us tour of up...,-rCqVHSxxfNSCBLvFE_U6Q
4,04UD14gamNjLY0IDYVhHJg,I am a long term frequent customer of this est...,1eKNwiFMPTLfviad0Sh-Ew


In [7]:
df.shape

(4724684, 3)

In [8]:
df['photo_id'].isnull().sum()

4709621

In [9]:
df = df.dropna()

In [10]:
df.shape

(15063, 3)

In [11]:
df.head()

,business_id,text,photo_id
3,e4Vwtrqf-wpJfwesgvdgxQ,Cute interior and owner (?) gave us tour of up...,-rCqVHSxxfNSCBLvFE_U6Q
4,04UD14gamNjLY0IDYVhHJg,I am a long term frequent customer of this est...,1eKNwiFMPTLfviad0Sh-Ew
6,gebiRewfieSdtt17PTW6Zg,Had a party of 6 here for hibachi. Our waitres...,NML2x1GKH-ZrDLMWgljdTw
7,EQ-TZ2eeD_E0BHuvoaeG5Q,"Locals recommended Milktooth, and it's an amaz...",6ZCkhDPDjoNCi0khcHIJxw
8,lj-E32x9_FA7GmUrBGBEWg,Love going here for happy hour or dinner! Gre...,bEXUuGQrQftJyMoYFGlwog


In [16]:
df['text'].iloc[2]

"Had a party of 6 here for hibachi. Our waitress brought our separate sushi orders on one plate so we couldn't really tell who's was who's and forgot several items on an order. I understand making mistakes but the restaraunt was really quiet so we were kind of surprised. Usually hibachi is a fun lively experience and our  cook  said maybe three words, but he cooked very well his name was Francisco. Service was fishy, food was pretty good, and im hoping it was just an off night here. But for the money I wouldn't go back."

We can clearly see that this is a party place. We will not assess if the food is good or bad. Fornow just the atmosphere.

In [13]:
df.shape

(15063, 3)

In [14]:
df['business_id'].duplicated().sum()

0

# Let's add the atmosphere

In [22]:
def parse_ambience_from_file_line_by_line(file_path):
    business_ambience = {}
    with open(file_path, 'r', encoding='utf-8') as file:
        # Process each line (which is a separate JSON object) individually
        for line in file:
            try:
                entry = json.loads(line)
                business_id = entry['business_id']
                if 'attributes' in entry and entry['attributes'] and 'Ambience' in entry['attributes'] and entry['attributes']['Ambience']:
                    ambience_str = entry['attributes']['Ambience']
                    # Handle ambience string safely
                    try:
                        ambience_dict = eval(ambience_str)
                        true_ambiences = [k for k, v in ambience_dict.items() if v]
                        business_ambience[business_id] = '/'.join(true_ambiences)
                    except Exception as e:
                        print(f"Error parsing Ambience for business_id {business_id}: {e}")
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line: {e}")
    return business_ambience

In [24]:
import json
file_path = '/content/drive/MyDrive/yelp_academic_dataset_business.json'
business_ambience_mapping = parse_ambience_from_file_line_by_line(file_path)

Error parsing Ambience for business_id CF33F8-E6oudUQ46HnavjQ: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id fNssGWMjae-SUzyyGblspQ: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id yFqq66GxRjnhZXPgjA6hnw: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id idRTl_xdqB12GWXIsisW-g: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id kgMEBZG6rjkGeFzPaIM4MQ: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id hWETQd1m9UVNGyQ-iJhIbA: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id fsPqRDDG2igV63nP89xWUA: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id 0xe81EigRw_b5W8tg0Ca6Q: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id la1k3Qq-cUBZrTXyElb8QA: 'NoneType' object has no attribute 'items'
Error parsing Ambience for business_id at82YGu

In [25]:
business_ambience_mapping

{'k0hlBqXX-Bt0vf1op7Jr1w': '',
 'eEOYSgkmpB90uNA7lDOMRA': '',
 'il_Ro8jwPlHresjw9EGmBg': 'casual',
 '0bPLkL0QhhPO5kt1_EXmNQ': 'casual',
 'MUTTqe8uqyMdBl186RmNeA': 'classy',
 'ROeacJQwBeh05Rqg7F6TCg': 'casual',
 'WKMJwqnfZKsAae75RMP6jA': 'hipster/trendy',
 '9OG5YkX1g2GReZM0AskizA': 'casual',
 '4iRzR7OaS-QaSXuvYxEGKA': '',
 'tMkwHmWFUEXrC9ZduonpTg': 'classy',
 'QdN72BWoyFypdGJhhI5r7g': 'hipster/casual',
 'Mjboz24M9NlBeiOJKLEd_Q': '',
 'kV_Q1oqis8Qli8dUoGpTyQ': 'casual',
 'aPNXGTDkf-4bjhyMBQxqpQ': 'classy',
 '2xVsWBNFwZOxIOdd9Mwnww': '',
 'ljxNT9p0y7YMPx0fcNBGig': 'classy/casual',
 'wghnIlMb_i5U46HMBGx9ig': 'casual',
 'lk9IwjZXqUMqqOhM774DtQ': 'trendy/classy/casual',
 'cVBxfMC4lp3DnocjYA3FHQ': '',
 'uI9XODGY_2_ieTE6xJ0myw': 'casual',
 'JgpnXv_0XhV3SfbfB50nxw': 'casual',
 'Si6LXSR9gvAbmgO5DzV2cw': '',
 'seKihQKpGGnCeLuELRQPSQ': 'casual',
 'L_TT0BFmFwORAMaA83K54A': 'casual',
 'qfWJmJ0g96eM_fWma3ja0g': 'casual',
 '8rb-3VYXE37IZix4yOdskw': 'divey',
 'aCDY7vXYMs54EbYuQScsnQ': '',
 'pJfh3Ct8iL5

In [26]:
df['LabelledAtmosphere'] = df['business_id'].map(business_ambience_mapping)


In [27]:
df.head()

,business_id,text,photo_id,LabelledAtmosphere
3,e4Vwtrqf-wpJfwesgvdgxQ,Cute interior and owner (?) gave us tour of up...,-rCqVHSxxfNSCBLvFE_U6Q,
4,04UD14gamNjLY0IDYVhHJg,I am a long term frequent customer of this est...,1eKNwiFMPTLfviad0Sh-Ew,casual
6,gebiRewfieSdtt17PTW6Zg,Had a party of 6 here for hibachi. Our waitres...,NML2x1GKH-ZrDLMWgljdTw,casual/classy
7,EQ-TZ2eeD_E0BHuvoaeG5Q,"Locals recommended Milktooth, and it's an amaz...",6ZCkhDPDjoNCi0khcHIJxw,hipster/classy
8,lj-E32x9_FA7GmUrBGBEWg,Love going here for happy hour or dinner! Gre...,bEXUuGQrQftJyMoYFGlwog,classy


In [28]:
df.to_csv("/content/drive/MyDrive/dataset_labelled.csv", index=False)

# EDA

In [29]:
df = pd.read_csv("/content/drive/MyDrive/dataset_labelled.csv")

In [30]:
df.head()

,business_id,text,photo_id,LabelledAtmosphere
0,e4Vwtrqf-wpJfwesgvdgxQ,Cute interior and owner (?) gave us tour of up...,-rCqVHSxxfNSCBLvFE_U6Q,NaN
1,04UD14gamNjLY0IDYVhHJg,I am a long term frequent customer of this est...,1eKNwiFMPTLfviad0Sh-Ew,casual
2,gebiRewfieSdtt17PTW6Zg,Had a party of 6 here for hibachi. Our waitres...,NML2x1GKH-ZrDLMWgljdTw,casual/classy
3,EQ-TZ2eeD_E0BHuvoaeG5Q,"Locals recommended Milktooth, and it's an amaz...",6ZCkhDPDjoNCi0khcHIJxw,hipster/classy
4,lj-E32x9_FA7GmUrBGBEWg,Love going here for happy hour or dinner! Gre...,bEXUuGQrQftJyMoYFGlwog,classy


In [39]:
df['LabelledAtmosphere'].value_counts(ascending=False).head(50)

casual                                     5603
classy                                     1154
classy/casual                              1141
trendy/casual                               296
trendy/classy                               256
trendy                                      176
trendy/classy/casual                        176
hipster/casual                              172
casual/trendy                               163
divey                                       138
divey/casual                                138
hipster/trendy/casual                       124
romantic/classy                             111
upscale/classy                               89
hipster                                      82
intimate/classy                              82
hipster/casual/trendy                        66
casual/classy                                65
intimate                                     60
hipster/trendy/classy/casual                 59
touristy/casual                         

In [ ]:
from transformers import pipeline

# Load a sentiment analysis pipeline
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# Example text
text = "The ambiance of the restaurant was cozy and inviting, with dim lighting and soft music."

# Predict the sentiment
result = classifier(text)

print(result)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

[{'label': '4 stars', 'score': 0.3810330629348755}]


In [ ]:
df['text'].iloc[6]

"Tremendous service (Big shout out to Douglas) that complemented the delicious food. Pretty expensive establishment (40-50$ avg for your main course), but its definitely backs that up with an atmosphere that's comparable with any of the top tier restaurants across the country."

In [ ]:
df['text'].iloc[8]

"I was really between 3 and 4 stars for this one. I LOVE the 96th street Naked Tchopstix so I was very excited to see this one which is closer to my house.\nThe vibe is totally different as this is geared more to take out although they do have a decent size dining area. You order at the counter and they deliver it.\nMy daughter and I tried the sushi bowl. You pick up a piece of paper and  select from a choice of proteins, vegetables, rice and toppings. I like the fact that it is on paper and you don't have to guess what things are or have to shout over the counter to tell the server what you want. The sushi bowl was good but not wonderful. The choices particularly of the vegetables seemed a little too limiting and I felt I had more rice than other food items. The sauces were tasty. My husband ordered the Maui roll which is my favorite roll at the 96th street location and this just was not up that standard. It was good but the quality and consistently was not as good as at 96th street. 

In [ ]:
df['text'].iloc[9]

'Went for lunch. Beef brisket sandwich was awesome.   So juicy and tender.    Pulled pork was was just as good!'

In [ ]:
df['photo_id'].iloc[9]

'7ZBZG-O2PZ3HrJaHguGEYQ'

In [ ]:
df['text'].iloc[10]

'Best thai food in the area.  Everything was authentic and delicious.  Will definitely be back again and again.'

In [ ]:
df['text'].iloc[15]

'What a great addition to the Funk Zone!  Grab a bite, grab some tastings, life is good. Right next door to the Santa Barbara Wine Collective, in fact it actually shares the same tables.  We had a fabulous savory croissant.'

In [ ]:
df['photo_id'].iloc[15]

'sRHnMbvjCIhxs_Yss33ENQ'